<a href="https://colab.research.google.com/github/takyaC/Finance/blob/main/EstimateAndActual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yahoo_fin

In [ ]:
pip install pandas

In [ ]:
pip install requests_html

In [47]:
from yahoo_fin.stock_info import get_analysts_info, get_cash_flow, get_income_statement, get_earnings_history, get_earnings, get_next_earnings_date
import pandas

In [107]:
tickers = [
 "AAPL","AFRM","AMZN","ANTM","APPS","BMY","BNTX","CCL","CRWD","DAL",
#  "DKNG","DOCU","EBS","FUTU","FVRR","GE","GOOGL","IAC","JPM","MRNA",
#  "MSFT","NIO","ORCL","PINS","PLUG","PTON","RBLX","RKT","ROKU","ROOT",
#  "RPRX","SBLK","SQ","TSLA","U","WKHS","WORK","ZI","ZM"
 ]

In [108]:
data_of_ticker = []

for ticker in tickers:
    try:
      # アナリスト予想（予想EPS、予想売上）
      ai = get_analysts_info(ticker)
      earnings_estimate = ai["Earnings Estimate"].iloc[[1], [1, 3, 4]]
      revenue_estimate = ai["Revenue Estimate"].iloc[[1], [1, 3, 4]]

      # EPS履歴
      earings = get_earnings(ticker)
      quarterly_results_row = len(earings["quarterly_results"]) -1
      earings_last_results = earings["quarterly_results"].iloc[quarterly_results_row, 2] # 右下のデータ(最終？Q、実績)を取得      

      # 損益計算書（四半期売上）
      ist = get_income_statement(ticker, False)
      total_revenue = ist.loc[["totalRevenue"], ist.columns[0]]

      # 単位付与(100万/１０億)
      total_revenue_by_million = total_revenue.values / 1000 / 1000
      if total_revenue_by_million < 1000:
        converted_total_revenue = "".join(str(total_revenue_by_million))[1:-1] + "M"
      else:
        converted_total_revenue = "".join(str(total_revenue_by_million / 1000))[1:-1] + "B"
      
      # 損益計算書（年間売上）
      isty = get_income_statement(ticker)
      total_revenue_yearly = isty.loc[["totalRevenue"], isty.columns[0]]

      # 単位付与(100万/１０億)
      total_revenue_yearly_by_million = total_revenue_yearly.values / 1000 / 1000
      if total_revenue_yearly_by_million < 1000:
        converted_total_revenue_yearly_by_million = "".join(str(total_revenue_yearly_by_million))[1:-1] + "M"
      else:
        converted_total_revenue_yearly_by_million = "".join(str(total_revenue_yearly_by_million / 1000))[1:-1] + "B"

      # キャッシュフロー計算書（純利益、営業キャッシュフロー、リスク（純利益<営業キャッシュフローが正常)）
      cfy = get_cash_flow(ticker)
      net_income_yearly = cfy.loc[["netIncome"], cfy.columns[0]]
      operating_cash_flow_yearly = cfy.loc[["totalCashFromOperatingActivities"], cfy.columns[0]]
      is_risk_cleard  = "OK" if operating_cash_flow_yearly.values > net_income_yearly.values else "NG"

      # 営業キャッシュフローマージン (15%以上推奨)
      cash_flow_margin_yearly_before = (operating_cash_flow_yearly.values / total_revenue_yearly.values)
      cash_flow_margin_yearly = round(cash_flow_margin_yearly_before[0], 2)

      # 次回決算発表日
      next_earnings_date = get_next_earnings_date(ticker)

      # 設定
      data = []
      data.extend([ticker])

      data.extend([earings_last_results]) # 前四半期 EPS
      data.extend([converted_total_revenue]) # 前四半期 売上
      data.extend([earnings_estimate.values[0][0]]) # 今四半期予想 EPS
      data.extend([revenue_estimate.values[0][0]]) # 今四半期予想 売上
      data.extend([earnings_estimate.values[0][1]]) # 今年度予想 EPS
      data.extend([revenue_estimate.values[0][1]]) # 今年度予想 売上
      data.extend([earnings_estimate.values[0][2]]) # 次年度予想 EPS
      data.extend([revenue_estimate.values[0][2]]) # 次年度予想 売上
      # data.extend([converted_total_revenue_yearly_by_million]) # 前年度売上

      data.extend([is_risk_cleard])
      data.extend([cash_flow_margin_yearly])
      
      data.extend(["-"]) # 空列
      data.extend([next_earnings_date]) # 次回決算発表日
      data_of_ticker.append(data)

      print(data)
    except Exception as e:
      print(e)
      data_of_ticker.append([ticker])
      

--------------------
--------------------
['AAPL', 1.41, '111.439B', 0.98, '76.92B', 4.45, '333.58B', 4.68, '348.17B', 'OK', 0.29, '-', datetime.datetime(2021, 4, 28, 0, 0)]
No tables found
--------------------
--------------------
['AMZN', 7.23, '125.555B', 9.37, '104.41B', 47.68, '473.93B', 66.11, '557.08B', 'OK', 0.17, '-', datetime.datetime(2021, 4, 28, 0, 0)]
--------------------
--------------------
['ANTM', 2.52, '31.825B', 6.35, '32.91B', 24.81, '134.96B', 28.15, '144.69B', 'OK', 0.09, '-', datetime.datetime(2021, 4, 21, 0, 0)]
--------------------
--------------------
['APPS', 0.17, '88.592M', 0.18, '82.4M', 0.68, '300.9M', 0.96, '392.63M', 'OK', 0.23, '-', datetime.datetime(2021, 2, 3, 0, 0)]
--------------------
--------------------
['BMY', 1.41, '11.068B', 1.82, '11.17B', 7.48, '46.32B', 8.05, '48.32B', 'OK', 0.33, '-', datetime.datetime(2021, 4, 29, 0, 0)]
--------------------
--------------------
['BNTX', -0.45, '67.458M', -0.18, '227.18M', -1.84, '488.27M', 17.59, '8.01B

In [109]:
pandas.DataFrame(data_of_ticker, columns=[
        "銘柄",
        "前四半期EPS",
        "前四半期売上",
        "今四半期予想　EPS", 
        "今四半期予想　売上",
        "今年度予想　EPS", 
        "今年度予想　売上",
        "次年度予想　EPS",
        "次年度予想　売上",
        "リスク",
        "営業キャッシュフローマージン",
        "-",
        "次回決算発表日"])


,銘柄,前四半期EPS,前四半期売上,今四半期予想 EPS,今四半期予想 売上,今年度予想 EPS,今年度予想 売上,次年度予想 EPS,次年度予想 売上,リスク,営業キャッシュフローマージン,-,次回決算発表日
0,AAPL,1.41,111.439B,0.98,76.92B,4.45,333.58B,4.68,348.17B,OK,0.29,-,2021-04-28
1,AFRM,NaN,None,NaN,None,NaN,None,NaN,None,None,NaN,None,NaT
2,AMZN,7.23,125.555B,9.37,104.41B,47.68,473.93B,66.11,557.08B,OK,0.17,-,2021-04-28
3,ANTM,2.52,31.825B,6.35,32.91B,24.81,134.96B,28.15,144.69B,OK,0.09,-,2021-04-21
4,APPS,0.17,88.592M,0.18,82.4M,0.68,300.9M,0.96,392.63M,OK,0.23,-,2021-02-03
5,BMY,1.41,11.068B,1.82,11.17B,7.48,46.32B,8.05,48.32B,OK,0.33,-,2021-04-29
6,BNTX,-0.45,67.458M,-0.18,227.18M,-1.84,488.27M,17.59,8.01B,NG,-1.83,-,2020-11-10
7,CCL,-1.94,33.0M,-1.55,208.28M,-4.02,5.76B,0.23,18.09B,OK,-1.13,-,2021-03-18
8,CRWD,0.00,232.46M,0.08,250.44M,0.22,860.17M,0.28,1.22B,OK,0.21,-,2021-03-16
9,DAL,-2.50,3.973B,-2.71,4.06B,-2.76,25.71B,3.83,37.86B,OK,-0.22,-,2021-04-20
